In [1]:
import seaborn as sns
import collections as col
import pandas as pd
import statistics
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Union
from pathlib import Path
import gseapy as gp
import functions_sheet as myfunc
from gseapy import barplot, dotplot
import re
import upsetplot as usp
from collections import Counter
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
from gseapy import Msigdb
from gseapy import GSEA

path_to_main_comparison_dir = "/nfs/data3/IHEC/TF_PRIO/comparisons/cells"
path_to_all_tfs_file = "/nfs/data3/IHEC/TF_PRIO/additional_files/all_tfs.txt"
path_to_figure_folder = "/nfs/data3/IHEC/TF_PRIO/created_plots/cellgroup_specific_TFs"

In [42]:
tf_tpm_df = myfunc.create_overlap_df_with_tpm(path_to_main_comparison_dir, tpm_filter=1)
tf_tpm_df = tf_tpm_df.rename(columns={"euchromatin": "active_markers", "heterochromatin": "suppressive_markers"})

In [44]:
def extract_conditions(row_name):
    cell_types = ["b-cells-healthy", "macrophage-healthy", "t-cell-healthy",  "monocyte-healthy", "neutrophil-healthy", "b-cells-leukemia"]
    for cell_type_1 in cell_types:
        if cell_type_1 in row_name:
            cell_types.remove(cell_type_1)
            for cell_type_2 in cell_types:
                if cell_type_2 in row_name:
                    return pd.Series([cell_type_1, cell_type_2])

In [45]:
tf_tpm_df[["celltype_1", "celltype_2"]] = tf_tpm_df.index.to_series().apply(extract_conditions)

# first attempt to filter for cellgroup specific TFs

In [ ]:
cell_types = ["b-cells-healthy", "macrophage-healthy", "t-cell-healthy",  "monocyte-healthy", "neutrophil-healthy"]

tf_dict_tpm_active_temp = tf_overlap_df_tpm_changed[tf_overlap_df_tpm_changed.index.str.contains("monocyte-healthy") 
    | tf_overlap_df_tpm_changed.index.str.contains("macrophage-healthy") 
    | tf_overlap_df_tpm_changed.index.str.contains("neutrophil-health") 
    & ~tf_overlap_df_tpm_changed.index.str.contains("b-cells-leukemia")]["active_markers"].to_dict()
tf_dict_tpm_suppressive_temp = tf_overlap_df_tpm_changed[tf_overlap_df_tpm_changed.index.str.contains("monocyte-healthy") 
    | tf_overlap_df_tpm_changed.index.str.contains("macrophage-healthy") 
    | tf_overlap_df_tpm_changed.index.str.contains("neutrophil-health") 
    & ~tf_overlap_df_tpm_changed.index.str.contains("b-cells-leukemia")]["suppressive_markers"].to_dict()
myfunc.upset_plot(tf_dict_tpm_active_temp, title_extension="Myeloid cell-active_markers", min_subset_size=0,max_subset_size=0)   
myfunc.upset_plot(tf_dict_tpm_suppressive_temp, title_extension="Myeloid cell-suppressive_markers", min_subset_size=0,max_subset_size=0)   

In [46]:
#lymphoid cells
lymphoid_cells_tf_df = tf_tpm_df[tf_tpm_df.index.str.contains("b-cells-healthy") 
    | tf_tpm_df.index.str.contains("t-cell-healthy") 
    & ~tf_tpm_df.index.str.contains("b-cells-leukemia")]

lymphoid_cells_active_all_shared = lymphoid_cells_tf_df["active_markers"].tolist()
lymhpoid_cells_suppressive_all_shared = lymphoid_cells_tf_df["suppressive_markers"].tolist()
print(set.intersection(*lymhpoid_cells_active_all_shared))
print(set.intersection(*lymhpoid_cells_suppressive_all_shared))

{'JUN', 'FOSL2'}
set()


In [47]:
#myeloid cells
myeloid_cells_tf_df = tf_tpm_df[tf_tpm_df.index.str.contains("macrophage-healthy") 
    | tf_tpm_df.index.str.contains("monocyte-healthy") 
    | tf_tpm_df.index.str.contains("neutrophil-health") 
    & ~tf_tpm_df.index.str.contains("b-cells-leukemia")]

myeloid_cells_tf_active_all_shared = myeloid_cells_tf_df["active_markers"].tolist()
myeloid_cells_tf_suppressive_all_shared = myeloid_cells_tf_df["suppressive_markers"].tolist()

print(set.intersection(*myeloid_cells_tf_active_all_shared))
print(set.intersection(*myeloid_cells_tf_suppressive_all_shared))

set()
set()


In [40]:
myeloid_cells_tf_df

,suppressive_markers,active_markers,intersection,union,unique,shared_at_least_once,shared_with_all
macrophage-healthy-neutrophil-healthy,"{ZNF384, JUN, MBD2, VENTX, NR2C1, ZNF460, STAT...","{MBD2, FOXO3, KLF10, CEBPA, IRF2, NR2C2, ARNT:...","{MBD2, ATF1, ZNF460, STAT5A, NFE2L1, SREBF1, Z...","{ZNF384, MBD2, NR2C1, ZNF460, SREBF1, FOXO3, Z...",{},"{ZNF384, MBD2, NR2C1, FOXO3, KLF10, CEBPA, IRF...",{}
macrophage-healthy-monocyte-healthy,"{ZNF384, NR1H3, NR2C1, SP2, MEF2C, STAT5A, MAX...","{FOSB, ZNF281, PBX2, MBD2, NR2C1, STAT5A, SREB...","{VEZF1, NR2C1, MECP2, STAT5A, GMEB2, PRDM4, RE...","{FOSB, ZNF384, NR1H3, ZNF281, MBD2, NR2C1, SRE...","{ZNF281, DBP, SOX12, HES6, ZNF701, ZBTB18}","{FOSB, ZNF384, NR1H3, PBX2, MBD2, NR2C1, SP2, ...",{}
macrophage-healthy-t-cell-healthy,"{NR1H3, NR4A2, KLF10, IRF2, NR2C2, ZNF189, ZNF...","{ZNF384, TCF7, NR2C1, SP2, STAT5A, NFE2L1, FOX...","{TCF7, VEZF1, JUN::JUNB, NFE2L1, HIC2, NR4A2, ...","{TCF7, NR1H3, ZNF384, NR2C1, SREBF1, FOXO3, ZF...",{},"{ZNF384, NR1H3, NR2C1, FOXO3, NR4A2, KLF10, IR...",{}
b-cells-healthy-monocyte-healthy,"{ETS1, NR2C1, ZNF460, STAT5A, NFE2L1, FOXO3, Z...","{MBD2, NR2C1, FOXO3, KLF10, IRF2, E2F6, ARNT::...","{VEZF1, ETS1, ZNF460, NR2C1, FOXO3, ZFX, NFE2L...","{MBD2, NR2C1, ZNF460, SREBF1, FOXO3, ZFX, ZEB1...",{ELF2},"{MBD2, NR2C1, FOXO3, KLF10, IRF2, E2F6, ARNT::...",{}
monocyte-healthy-neutrophil-healthy,"{ZNF384, PBX2, NR2C1, SP2, NFE2L1, VDR, FOXJ2,...","{NR2C1, FOXO3, NR4A2, IRF2, E2F6, ARNT::HIF1A,...","{PBX2, NR2C1, SP2, STAT2, NFE2L1, TCF12, FOXJ2...","{ZNF384, NR2C1, ZNF460, SREBF1, FOXO3, ZFX, NR...",{},"{ZNF384, NR2C1, FOXO3, NR4A2, CEBPA, IRF2, E2F...",{}
t-cell-healthy-neutrophil-healthy,"{FOSB, ZNF384, EOMES, PBX2, STAT5A, NFE2L1, SR...","{FOSB, ZNF384, ZNF148, SREBF1, MAX, FOXJ2, NR4...","{FOSB, ZNF384, ZNF354C, MAZ, ZBTB17, MGA, KLF6...","{FOSB, ZNF384, SREBF1, NR4A2, IRF2, NR2C2, MEF...","{JUNB, STAT1}","{FOSB, ZNF384, NR4A2, IRF2, NR2C2, MEF2A, ARNT...",{}
b-cells-healthy-macrophage-healthy,"{FOSB, ZNF384, NR4A2, KLF10, IRF2, MEF2A, ARNT...","{JUN, NR2C1, NFE2L1, POU3F1, ZFX, FOXO3, VDR, ...","{FOSL2, POU3F1, NFE2L2, ZFX, HIC2, VDR, FOXJ2,...","{FOSB, ZNF384, NR2C1, POU3F1, SREBF1, ZFX, FOX...",{ZFP14},"{FOSB, ZNF384, NR2C1, FOXO3, NR4A2, KLF10, IRF...",{}
monocyte-healthy-t-cell-healthy,"{TCF7, ZNF460, NFE2L1, VDR, ZNF317, ARID5B, AR...","{ZNF384, TCF7, ZKSCAN1, MBD2, VENTX, ATF1, SP2...","{TCF7, STAT2, NFE2L1, NFE2L2, SREBF2, VDR, BCL...","{TCF7, ZNF384, ZKSCAN1, MBD2, ZNF460, SREBF1, ...","{ZKSCAN1, STAT1::STAT2}","{ZNF384, MBD2, KLF10, IRF2, E2F6, MEF2A, ARNT:...",{}
b-cells-healthy-neutrophil-healthy,"{FOSB, ZNF384, NR4A2, NR2C2, MEF2A, ARNT::HIF1...","{ZNF341, ZNF148, CEBPG, NR2C1, SP2, STAT5A, NR...","{ZNF341, SP2, STAT5A, NR4A2, FOXJ2, MAFK, KLF1...","{FOSB, ZNF384, TCF7, CEBPG, NR2C1, ZNF460, POU...","{CEBPG, SPI1, EGR1, PLAG1}","{FOSB, ZNF384, NR2C1, NR4A2, CEBPA, NR2C2, MEF...",{}


In [48]:
myeloid_cells_tf_df

,suppressive_markers,active_markers,celltype_1,celltype_2
macrophage-healthy-neutrophil-healthy,"{ZNF384, JUN, MBD2, VENTX, NR2C1, ZNF460, STAT...","{MBD2, FOXO3, KLF10, CEBPA, IRF2, NR2C2, ARNT:...",macrophage-healthy,neutrophil-healthy
macrophage-healthy-monocyte-healthy,"{ZNF384, NR1H3, NR2C1, SP2, MEF2C, STAT5A, MAX...","{FOSB, ZNF281, PBX2, MBD2, NR2C1, STAT5A, SREB...",macrophage-healthy,monocyte-healthy
macrophage-healthy-t-cell-healthy,"{NR1H3, NR4A2, KLF10, IRF2, NR2C2, ZNF189, ZNF...","{ZNF384, TCF7, NR2C1, SP2, STAT5A, NFE2L1, FOX...",macrophage-healthy,t-cell-healthy
b-cells-healthy-monocyte-healthy,"{ETS1, NR2C1, ZNF460, STAT5A, NFE2L1, FOXO3, Z...","{MBD2, NR2C1, FOXO3, KLF10, IRF2, E2F6, ARNT::...",b-cells-healthy,monocyte-healthy
monocyte-healthy-neutrophil-healthy,"{ZNF384, PBX2, NR2C1, SP2, NFE2L1, VDR, FOXJ2,...","{NR2C1, FOXO3, NR4A2, IRF2, E2F6, ARNT::HIF1A,...",monocyte-healthy,neutrophil-healthy
t-cell-healthy-neutrophil-healthy,"{FOSB, ZNF384, EOMES, PBX2, STAT5A, NFE2L1, SR...","{FOSB, ZNF384, ZNF148, SREBF1, MAX, FOXJ2, NR4...",t-cell-healthy,neutrophil-healthy
b-cells-healthy-macrophage-healthy,"{FOSB, ZNF384, NR4A2, KLF10, IRF2, MEF2A, ARNT...","{JUN, NR2C1, NFE2L1, POU3F1, ZFX, FOXO3, VDR, ...",b-cells-healthy,macrophage-healthy
monocyte-healthy-t-cell-healthy,"{TCF7, ZNF460, NFE2L1, VDR, ZNF317, ARID5B, AR...","{ZNF384, TCF7, ZKSCAN1, MBD2, VENTX, ATF1, SP2...",t-cell-healthy,monocyte-healthy
b-cells-healthy-neutrophil-healthy,"{FOSB, ZNF384, NR4A2, NR2C2, MEF2A, ARNT::HIF1...","{ZNF341, ZNF148, CEBPG, NR2C1, SP2, STAT5A, NR...",b-cells-healthy,neutrophil-healthy


->  problem: with this method comparisons are included where e.g. both used celltypes are from the same group

# second try

In [50]:
myeloid_cells = ["macrophage-healthy", "neutrophil-healthy", "monocyte-healthy"]
lymphoid_cells = ["b-cells-healthy", "t-cell-healthy"]


In [51]:
lymphoid_vs_myeloid_cells_tf_df = tf_tpm_df[((tf_tpm_df["celltype_1"].isin(myeloid_cells) & tf_tpm_df["celltype_2"].isin(lymphoid_cells)) | 
                                            (tf_tpm_df["celltype_1"].isin(lymphoid_cells) & tf_tpm_df["celltype_2"].isin(myeloid_cells)))]

In [52]:
lymphoid_vs_myeloid_cells_tf_df

,suppressive_markers,active_markers,celltype_1,celltype_2
macrophage-healthy-t-cell-healthy,"{NR1H3, NR4A2, KLF10, IRF2, NR2C2, ZNF189, ZNF...","{ZNF384, TCF7, NR2C1, SP2, STAT5A, NFE2L1, FOX...",macrophage-healthy,t-cell-healthy
b-cells-healthy-monocyte-healthy,"{ETS1, NR2C1, ZNF460, STAT5A, NFE2L1, FOXO3, Z...","{MBD2, NR2C1, FOXO3, KLF10, IRF2, E2F6, ARNT::...",b-cells-healthy,monocyte-healthy
t-cell-healthy-neutrophil-healthy,"{FOSB, ZNF384, EOMES, PBX2, STAT5A, NFE2L1, SR...","{FOSB, ZNF384, ZNF148, SREBF1, MAX, FOXJ2, NR4...",t-cell-healthy,neutrophil-healthy
b-cells-healthy-macrophage-healthy,"{FOSB, ZNF384, NR4A2, KLF10, IRF2, MEF2A, ARNT...","{JUN, NR2C1, NFE2L1, POU3F1, ZFX, FOXO3, VDR, ...",b-cells-healthy,macrophage-healthy
monocyte-healthy-t-cell-healthy,"{TCF7, ZNF460, NFE2L1, VDR, ZNF317, ARID5B, AR...","{ZNF384, TCF7, ZKSCAN1, MBD2, VENTX, ATF1, SP2...",t-cell-healthy,monocyte-healthy
b-cells-healthy-neutrophil-healthy,"{FOSB, ZNF384, NR4A2, NR2C2, MEF2A, ARNT::HIF1...","{ZNF341, ZNF148, CEBPG, NR2C1, SP2, STAT5A, NR...",b-cells-healthy,neutrophil-healthy


In [56]:
myeloid_lymphoid_cells_tf_active_all_shared = lymphoid_vs_myeloid_cells_tf_df["active_markers"].tolist()
myeloid_lymphoid_cells_tf_suppressive_all_shared = lymphoid_vs_myeloid_cells_tf_df["suppressive_markers"].tolist()

all_shared_tf_active_markers = list(set.intersection(*myeloid_lymphoid_cells_tf_active_all_shared))
all_shared_tf_suppressive_markers = list(set.intersection(*myeloid_lymphoid_cells_tf_suppressive_all_shared))

print(all_shared_tf_active_markers)
print(all_shared_tf_suppressive_markers)

['JUN', 'FOSL2']
['HIC2']


Checking for a TPM value of at least 1 in the respective cell types

In [54]:
tpm_df = myfunc.get_tpm_filtered_genes_df(path_to_main_comparison_dir, tpm_filter=1)

In [59]:
myeloid_cells = ["macrophage-healthy", "neutrophil-healthy", "monocyte-healthy"]
lymphoid_cells = ["b-cells-healthy", "t-cell-healthy"]

JUN and FOSL2 might either have a TPM value of at least 1 on all lymhpoid cell types (B-cels, T-cells) and/or all myeloid cell types (macropahges, neutrophils, monocytes) or not fullfilling any of these both requirements

In [58]:
for TF in all_shared_tf_active_markers:
    if all(TF in tpm_df.loc[cell_type, "TPM_filtered_genes"] for cell_type in lymphoid_cells):
        print(TF +" positive for lymphoid cells")
    else:
        print("negative for lymphoid cells")

JUN positive for lymphoid cells
FOSL2 positive for lymphoid cells


In [61]:
for TF in all_shared_tf_active_markers:
    if all(TF in tpm_df.loc[cell_type, "TPM_filtered_genes"] for cell_type in myeloid_cells):
        print(TF +" positive for myeloid cells")
    else:
        print(TF +" negative for myeloid cells")

JUN positive for myeloid cells
FOSL2 positive for myeloid cells


In [64]:
for TF in all_shared_tf_suppressive_markers:
    if all(TF in tpm_df.loc[cell_type, "TPM_filtered_genes"] for cell_type in lymphoid_cells):
        print(TF +" positive for lymphoid cells")
    else:
        print(TF +" negative for lymphoid cells")

HIC2 positive for lymphoid cells


In [65]:
for TF in all_shared_tf_suppressive_markers:
    if all(TF in tpm_df.loc[cell_type, "TPM_filtered_genes"] for cell_type in myeloid_cells):
        print(TF +" positive for myeloid cells")
    else:
        print(TF +" negative for myeloid cells")

HIC2 negative for myeloid cells


In [60]:
print("JUN" in tpm_df.loc["b-cells-healthy", "TPM_filtered_genes"])
print("JUN" in tpm_df.loc["t-cell-healthy", "TPM_filtered_genes"])

True
True
